## Notebook to load catalog data, and perform the iteration

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import astropy.units as u
import gc
from astropy.coordinates import SkyCoord, GeocentricTrueEcliptic
from library.snr import optimal_snr
from library.lisa_psd import noise_psd_AE

/home/alice/miniforge3/envs/gwpy_env/lib/python3.12/site-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal
PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


## Step 1: pre-process the catalog

In the terminal, run:

`python preprocess_catalog.py     --filepath filepath  
   --output output_filenam.h5     --T_obs T_obs      --delta_t deltat      --tdi 1.5 or 2.0     --snr_preselection snr_preselection      --batch_size bach_size `

/home/alice/catalog_binaries/catalogue_interaction_WDWD_processed.hdf5

default parameters: T_obs = 126144000 (4 years), delta_t = 5, tdi = 1.5, snr_preselection = 0.01, batch_size = 1000


## Step 2: load processed data 

In [ ]:
from library.iteration_utils import load_waveforms, setup, run_iterative_separation

data = load_waveforms('Processed_data/processed_all_snr0001_1yrs.h5')

## Step 3: prepare state and parameters for the iteration and run it

Set SNR threshold and name of the output file with the resolved sources data. In `run_iterative_separation`, you can set the maximum number of iterations and the size of the median filter for the smoothing of the PSD, as well as turn on/off plots and saving the results.

In [ ]:
snr_thr = 7
results_filename = 'results_all_snr7_1yrs_sel0001'
T_obs = data['T_obs']
max_iterations = 50
filter_size = 2000

state = setup(data, snr_calculator= lambda source: optimal_snr(source["A"], source["psd_total"], T_obs=T_obs), 
              psd_instrumental=noise_psd_AE, 
              snr_threshold=snr_thr,
              filter_size=filter_size)
del data
gc.collect()
results = run_iterative_separation(state,  
                                   max_iterations=max_iterations, 
                                   filter_size=filter_size, 
                                   print_progress=True, 
                                   plot= False, 
                                   save_results=True, 
                                   output_file= results_filename)

Starting with 15539324 total sources
13132454 sources already pre-excluded and added to the confusion
2406870 candidate sources to run the iteration
Frequency range: 1.04e-04 to 2.04e-02 Hz
SNR threshold: 7

--- Step 0: Instrument-only SNR pre-exclusion
  Unresolvable sources (added to confusion): 2394715
  SNR candidates for iteration: 12155

--- Iteration 1 ---
  New resolved: 5077
  Still unresolved: 7078
  Total resolved: 5077

--- Iteration 2 ---
  New resolved: 30
  Still unresolved: 7048
  Total resolved: 5107

--- Iteration 3 ---
  New resolved: 0
  Still unresolved: 7048
  Total resolved: 5107

Converged after 3 iterations!

FINAL RESULTS:
  Total sources: 15539324
  Resolved: 5107 (0.0%)
  Unresolved: 15534217 (100.0%)

Saving resolved sources to results/results_all_snr7_4yrs_sel0001...
Results saved to results/results_all_snr7_4yrs_sel0001


In [4]:
results.keys()

dict_keys(['resolved_sources', 'global_fr', 'psd_confusion', 'unresolved_indices', 'n_resolved', 'n_unresolved', 'iterations', 'history'])

In [5]:
results["psd_confusion"]

[(1,
  array([1.06181056e-41, 1.06166081e-41, 1.06151109e-41, ...,
         6.36933197e-40, 6.36933887e-40, 6.36934577e-40], shape=(2557780,))),
 (2,
  array([1.06181056e-41, 1.06166081e-41, 1.06151109e-41, ...,
         6.36933197e-40, 6.36933887e-40, 6.36934577e-40], shape=(2557780,))),
 (3,
  array([1.06181056e-41, 1.06166081e-41, 1.06151109e-41, ...,
         6.36933197e-40, 6.36933887e-40, 6.36934577e-40], shape=(2557780,)))]

In [6]:
results["resolved_sources"][0]["source"].keys()

dict_keys(['f0', 'fdot', 'Ampl', 'fr', 'A', 'psd_total', 'id', 'ecliptic_lon', 'ecliptic_lat'])